In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
import math
import datetime
from Tools import *

In [3]:
def format_time(dfTime):
    x=[]
    for i in dfTime:
        i=str(i)
        new_i=""
        count=1
        for j in i:
            if count%2==0 and count!=len(i):
                new_i+=j
                new_i+=":"
            else:
                new_i+=j
            count+=1
        x.append(new_i)
    xy=pd.Series(x)
    return xy

In [4]:
def add_minute(df):
    from datetime import date
    from dateutil.relativedelta import relativedelta
    
    c=pd.to_datetime(df)
    after6M=c+relativedelta(minutes=+6)
    befor6M=c+relativedelta(minutes=-6)
    return after6M,befor6M


In [5]:
def add_minute2(df):
    from datetime import date
    from dateutil.relativedelta import relativedelta
    minutes=[]

    c=pd.to_datetime(df)
    after6M=c+relativedelta(minutes=+6)
    #befor6M=c+relativedelta(minutes=-6)
    #minutes.append([after6M,befor6M,i])
    return after6M

Fiyatlar okunuyor

In [6]:
df=pd.read_csv("tsla.us.txt",sep=",",names=['TICKER','PER','DATE','TIME','OPEN','HIGH','LOW','CLOSE','VOL','OPENINT'])

Gereksiz kolonlar siliniyor. Date kolonu formatlanıyor

In [7]:
df=df.drop(["PER","TICKER"],axis='columns')
df["DATE2"]=pd.to_datetime(df["DATE"],format="%Y%m%d")
df["DATE2"]=df["DATE2"].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y"))
df["TIME2"]=format_time(df["TIME"])
df

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00
...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00


In [8]:
df.tail()

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00
2729,20201019,215500,432.60,432.85,430.410,430.830,1028440,0,19/10/2020,21:55:00


Günlük fiyat değişimleri hesaplanıyor

In [9]:
df["DIFF"]=df["OPEN"]-df["CLOSE"]
df.loc[df['DIFF'] < 0, 'STATUS'] = -1
df.loc[df['DIFF'] > 0, 'STATUS'] = 1
df.loc[df['DIFF'] == 0, 'STATUS'] = 0 ## TO-DO: Bir tolerans aralığındaki değerleri 0 olarak işaretlenmesi yapıladak.
df["DIFF_PERC"]=(df["DIFF"]*100)/df["OPEN"]
df

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2,DIFF,STATUS,DIFF_PERC
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00,-2.419,-1.0,-0.544072
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00,4.880,1.0,1.091625
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00,-10.069,-1.0,-2.277642
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00,-7.568,-1.0,-1.674929
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00,2.790,1.0,0.607578
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00,-1.055,-1.0,-0.244309
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00,0.880,1.0,0.203299
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00,-1.950,-1.0,-0.451389
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00,1.350,1.0,0.311096


Zaman kolunu formatlanıyor. DATETIME kolonu oluşturuluyor

In [10]:
df["DATETIME"]=df["DATE2"]+" "+df["TIME2"]
df["DATETIME"]=pd.to_datetime(df["DATETIME"])
df["DATETIME"]=df["DATETIME"].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y %H:%M:%S"))
df["DATETIME"]=pd.to_datetime(df["DATETIME"])
df

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2,DIFF,STATUS,DIFF_PERC,DATETIME
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00,-2.419,-1.0,-0.544072,2020-08-31 15:30:00
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00,4.880,1.0,1.091625,2020-08-31 15:35:00
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00,-10.069,-1.0,-2.277642,2020-08-31 15:40:00
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00,-7.568,-1.0,-1.674929,2020-08-31 15:45:00
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00,2.790,1.0,0.607578,2020-08-31 15:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00,-1.055,-1.0,-0.244309,2020-10-19 21:35:00
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00,0.880,1.0,0.203299,2020-10-19 21:40:00
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00,-1.950,-1.0,-0.451389,2020-10-19 21:45:00
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00,1.350,1.0,0.311096,2020-10-19 21:50:00


# Dosyalardaki verileri tek dosyada toplama- Bu kısım tek seferlik. Silinecek

In [11]:
df_news=pd.DataFrame()
for i in range(1,7):
    path="TeslaNews"+str(i)+".csv"
    temp_df=pd.read_csv(path)
    df_news=df_news.append(temp_df)

In [12]:
df_news.info()
df_news.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9536 entries, 0 to 1586
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9536 non-null   int64 
 1   Dates       9536 non-null   object
 2   Text        9536 non-null   object
 3   Link        9536 non-null   object
dtypes: int64(1), object(3)
memory usage: 372.5+ KB


,Unnamed: 0,Dates,Text,Link
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...


Gereksiz kolonlar silindi

In [13]:
df_news.drop(df_news.columns[df_news.columns.str.contains("Unnamed",case=False)],axis=1,inplace=True)

"ID" kolonu eklendi

In [14]:
df_news.insert(0,"ID",range(0,len(df_news)))

Date kolonu formatlanıyor

In [15]:
df_news['DATETIME'] = pd.to_datetime(df_news['Dates'])
#df_news['NewDates'] = df_news['NewDates'].dt.tz_localize('US/Eastern').dt.tz_convert('UTC')
df_news['DATETIME'] = df_news['DATETIME'].apply(lambda x: datetime.datetime.replace(x, tzinfo=None))
df_news

C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\ProgramData\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,ID,Dates,Text,Link,DATETIME
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,2020-10-13 07:41:00
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,2020-10-13 07:11:00
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,2020-10-13 07:06:00
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,2020-10-13 06:22:00
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,2020-10-13 05:28:00
...,...,...,...,...,...
1582,9531,"Apr 29, 2016 3:43AM EDT",Tesla CEO Elon Musk introduces the Model X. So...,https://www.nasdaq.com/articles/mark-zuckerber...,2016-04-29 03:43:00
1583,9532,"Apr 29, 2016 2:22AM EDT",Ford ( F ) reported an all time record pre-tax...,https://www.nasdaq.com/articles/ford-posts-rec...,2016-04-29 02:22:00
1584,9533,"Apr 28, 2016 11:44AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/alphabet-inc%3...,2016-04-28 11:44:00
1585,9534,"Apr 28, 2016 10:23AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/apple-inc.-aap...,2016-04-28 10:23:00


In [16]:
## TO-DO: Stock price EDT formatında mı? Kontrol edilecek.
df_news

,ID,Dates,Text,Link,DATETIME
0,0,"Oct 13, 2020 7:41AM EDT",Eikon search string for individual stock moves...,https://www.nasdaq.com/articles/buzz-u.s.-stoc...,2020-10-13 07:41:00
1,1,"Oct 13, 2020 7:11AM EDT","Back in 1993, when drugmaker Amarin's (NASDAQ:...",https://www.nasdaq.com/articles/the-top-2-cont...,2020-10-13 07:11:00
2,2,"Oct 13, 2020 7:06AM EDT",Electric vehicle pioneer Tesla (NASDAQ: TSLA) ...,https://www.nasdaq.com/articles/forget-tesla-i...,2020-10-13 07:06:00
3,3,"Oct 13, 2020 6:22AM EDT",Oct 13 (Reuters) - Tesla Inc TSLA.Osaid on Tue...,https://www.nasdaq.com/articles/tesla-cuts-pri...,2020-10-13 06:22:00
4,4,"Oct 13, 2020 5:28AM EDT","Adds details, backgroundBEIJING, Oct 13 (Reute...",https://www.nasdaq.com/articles/audi-to-launch...,2020-10-13 05:28:00
...,...,...,...,...,...
1582,9531,"Apr 29, 2016 3:43AM EDT",Tesla CEO Elon Musk introduces the Model X. So...,https://www.nasdaq.com/articles/mark-zuckerber...,2016-04-29 03:43:00
1583,9532,"Apr 29, 2016 2:22AM EDT",Ford ( F ) reported an all time record pre-tax...,https://www.nasdaq.com/articles/ford-posts-rec...,2016-04-29 02:22:00
1584,9533,"Apr 28, 2016 11:44AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/alphabet-inc%3...,2016-04-28 11:44:00
1585,9534,"Apr 28, 2016 10:23AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/apple-inc.-aap...,2016-04-28 10:23:00


Tekrarlı veriler hesaplanıp temizleniyor

In [17]:
#df_news["Dates"]=datetime.strptime(df_news["Dates"],"%d/%m/%Y")
#df_news['NewDates'] = df_news['NewDates'].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y %H:%M:%S"))


df_temp=df_news[df_news.duplicated(keep=False)]
print("Tüm kolonlar tekrarlı veri sayısı: ",df_temp.shape)
df_temp=df_news[df_news.duplicated(subset=["Text"],keep=False)]
print("Text kolonu  tekrarlı veri sayısı: ",df_temp.shape)
df_temp=df_temp.drop(["Link"],axis=1)
df_temp


Tüm kolonlar tekrarlı veri sayısı:  (0, 5)
Text kolonu  tekrarlı veri sayısı:  (100, 5)


,ID,Dates,Text,DATETIME
62,62,"Oct 6, 2020 9:10AM EDT","InvestorPlace - Stock Market News, Stock Advic...",2020-10-06 09:10:00
81,81,"Oct 4, 2020 9:00AM EDT","By David RandallNEW YORK, Oct 2 (Reuters) - In...",2020-10-04 09:00:00
89,89,"Oct 2, 2020 4:52PM EDT","By David RandallNEW YORK, Oct 2 (Reuters) - In...",2020-10-02 16:52:00
163,163,"Sep 29, 2020 2:00AM EDT","By Marc JonesLONDON, Sept 28 (Reuters) - Finan...",2020-09-29 02:00:00
166,166,"Sep 28, 2020 7:00PM EDT","By Marc JonesLONDON, Sept 28 (Reuters) - Finan...",2020-09-28 19:00:00
...,...,...,...,...
90,8039,"Feb 28, 2017 7:31AM EST","The stock market is, for lack of a better word...",2017-02-28 07:31:00
123,8072,"Feb 24, 2017 12:12PM EST",Shutterstock photoThe broad stock market outpa...,2017-02-24 12:12:00
124,8073,"Feb 24, 2017 12:12PM EST",Shutterstock photoThe broad stock market outpa...,2017-02-24 12:12:00
575,8524,"Nov 17, 2016 2:14AM EST","According to Trefis estimates, nearly half of ...",2016-11-17 02:14:00


In [18]:
df_news[df_news.duplicated(subset=["DATETIME"],keep=False)]

,ID,Dates,Text,Link,DATETIME
14,14,"Oct 9, 2020 2:12AM EDT","With the battery design, manufacturing, and in...",https://www.nasdaq.com/articles/tesla-can-post...,2020-10-09 02:12:00
15,15,"Oct 9, 2020 2:12AM EDT",Electric vehicle major Tesla (NASDAQ: TSLA) is...,https://www.nasdaq.com/articles/teslas-expense...,2020-10-09 02:12:00
86,86,"Oct 2, 2020 5:34PM EDT","By Jeff LewisTORONTO, Oct 2 (Reuters) - Brazil...",https://www.nasdaq.com/articles/miner-vale-in-...,2020-10-02 17:34:00
87,87,"Oct 2, 2020 5:34PM EDT","Just three months ago, car manufacturer Nikola...",https://www.nasdaq.com/articles/what-you-need-...,2020-10-02 17:34:00
91,91,"Oct 2, 2020 4:26PM EDT",Shares ofÂ LemonadeÂ (NYSE: LMND) were turning...,https://www.nasdaq.com/articles/why-lemonade-s...,2020-10-02 16:26:00
...,...,...,...,...,...
1401,9350,"Jun 10, 2016 9:30AM EDT","For Immediate ReleaseChicago, IL - June 10, 20...",https://www.nasdaq.com/articles/the-zacks-anal...,2016-06-10 09:30:00
1443,9392,"Jun 1, 2016 10:32AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/kors-stock%3A-...,2016-06-01 10:32:00
1444,9393,"Jun 1, 2016 10:32AM EDT",InvestorPlaceInvestorPlace - Stock Market New...,https://www.nasdaq.com/articles/tesla-motors-i...,2016-06-01 10:32:00
1459,9408,"May 26, 2016 11:45AM EDT","Shutterstock photoIn trading on Thursday, shar...",https://www.nasdaq.com/articles/tesla-motors-b...,2016-05-26 11:45:00


In [19]:
def functt(i):
    a=i*2
    b=i+3
    o=i+5
    return a,b,o

d=pd.DataFrame()

c=pd.Series([5,8,9])
d["number"]=c
f=pd.Series(["a","b","c"])
d["text"]=f


for i in range(0,len(d)):
    a=functt(d["number"][i])[0]
    d.loc[i,"AFTER"]=a
    b=functt(d["number"][i])[1]
    d.loc[i,"BEFORE"]=b
    c=functt(d["number"][i])[2]
    d.loc[i,"ORIGINALS"]=c
d

,number,text,AFTER,BEFORE,ORIGINALS
0,5,a,10.0,8.0,10.0
1,8,b,16.0,11.0,13.0
2,9,c,18.0,12.0,14.0


+- 6 dk eklendi.

In [20]:
for i in range(0,len(df)):
    after=add_minute(df["DATETIME"][i])[0]
    df.loc[i,"AFTER"]=after
    before=add_minute(df["DATETIME"][i])[1]
    df.loc[i,"BEFORE"]=before
df
    


,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL,OPENINT,DATE2,TIME2,DIFF,STATUS,DIFF_PERC,DATETIME,AFTER,BEFORE
0,20200831,153000,444.61,450.49,440.110,447.029,1766358,0,31/08/2020,15:30:00,-2.419,-1.0,-0.544072,2020-08-31 15:30:00,2020-08-31 15:36:00,2020-08-31 15:24:00
1,20200831,153500,447.04,447.80,441.210,442.160,661342,0,31/08/2020,15:35:00,4.880,1.0,1.091625,2020-08-31 15:35:00,2020-08-31 15:41:00,2020-08-31 15:29:00
2,20200831,154000,442.08,452.28,440.710,452.149,722964,0,31/08/2020,15:40:00,-10.069,-1.0,-2.277642,2020-08-31 15:40:00,2020-08-31 15:46:00,2020-08-31 15:34:00
3,20200831,154500,451.84,459.53,450.050,459.408,1570773,0,31/08/2020,15:45:00,-7.568,-1.0,-1.674929,2020-08-31 15:45:00,2020-08-31 15:51:00,2020-08-31 15:39:00
4,20200831,155000,459.20,461.00,455.500,456.410,1346307,0,31/08/2020,15:50:00,2.790,1.0,0.607578,2020-08-31 15:50:00,2020-08-31 15:56:00,2020-08-31 15:44:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,20201019,213500,431.83,433.04,431.700,432.885,334562,0,19/10/2020,21:35:00,-1.055,-1.0,-0.244309,2020-10-19 21:35:00,2020-10-19 21:41:00,2020-10-19 21:29:00
2726,20201019,214000,432.86,432.90,431.610,431.980,274050,0,19/10/2020,21:40:00,0.880,1.0,0.203299,2020-10-19 21:40:00,2020-10-19 21:46:00,2020-10-19 21:34:00
2727,20201019,214500,432.00,433.95,431.955,433.950,268830,0,19/10/2020,21:45:00,-1.950,-1.0,-0.451389,2020-10-19 21:45:00,2020-10-19 21:51:00,2020-10-19 21:39:00
2728,20201019,215000,433.95,434.12,432.240,432.600,380857,0,19/10/2020,21:50:00,1.350,1.0,0.311096,2020-10-19 21:50:00,2020-10-19 21:56:00,2020-10-19 21:44:00


In [21]:
df


df_news

for index,row in zip(df_news,df):
    t=row["DATETIME"]
    

TypeError: string indices must be integers